In [ ]:
!pip install pandarallel
!pip install mapply
!pip install parallel-pandas
!pip install swifter

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.DataFrame(np.random.randint(1, 10**6, size=(10**7, 4)), columns = list("ABCD"))
df.head()

,A,B,C,D
0,997705,76013,657149,947802
1,889105,961243,373884,808704
2,268708,997270,658497,547413
3,358699,318114,191243,640387
4,98992,509563,914493,785773


In [ ]:
df.shape

(10000000, 4)

In [ ]:
def sum_row(row):
    return sum(row)

In [ ]:
from os import cpu_count

n_cores = cpu_count()
n_cores

12

#### Нативный .apply

In [ ]:
%%time

res = df.apply(sum_row, axis=1)

CPU times: total: 1min 19s
Wall time: 1min 20s


#### Pandarallel

In [ ]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=n_cores)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [ ]:
%%time

res = df.parallel_apply(sum_row, axis=1)

CPU times: total: 2.3 s
Wall time: 37.5 s


#### Parallel-Pandas Apply

In [ ]:
from parallel_pandas import ParallelPandas

In [ ]:
ParallelPandas.initialize(n_cpu=n_cores, split_factor=4, disable_pr_bar=False)

In [ ]:
res = df.p_apply(sum_row, axis=1)

SUM_ROW DONE:   0%|          | 0/10000000 [00:00<?, ?it/s]

#### Mapply Apply

In [ ]:
import mapply

In [ ]:
mapply.init(
    n_workers=n_cores,
    chunk_size=100,
    max_chunks_per_worker=8,
    progressbar=True
)

In [ ]:
%%time

res = df.mapply(sum_row, axis=1)

Using more workers (12) than is sensible (7). For CPU-bound operations, consider lowering n_workers to avoid bottlenecks on the physical CPUs


  0%|                                                                                                         …

CPU times: total: 2.03 s
Wall time: 20.6 s


#### Swifter Apply

In [ ]:
import swifter

In [ ]:
%%time

res = df.swifter.apply(sum_row, axis=1)

Dask Apply:   0%|          | 0/25 [00:00<?, ?it/s]

CPU times: total: 4.61 s
Wall time: 33.7 s
